In [1]:
import os
os.environ["OPENAI_API_KEY"]=" "

In [3]:
# !pip install langchain_community

In [5]:
# !pip install langchain_openai

In [6]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 4.1 MB/s eta 0:00:00


In [7]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 20.0 MB/s eta 0:00:00


# Loading the Document into Database(FAISS-Facebook AI similarity search)
> For large data

In [9]:
import os
import logging
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# Configure logging
logging.basicConfig(level=logging.INFO)

if __name__ == "__main__":
    try:
        DB_FAISS_PATH = "vectorstore/db_faiss"
        PDF_PATH = "/content/Ramayanam@RAG.pdf"

        if not os.path.exists(PDF_PATH):
            raise FileNotFoundError(f"PDF file not found: {PDF_PATH}")

        logging.info("Loading PDF file...")
        loader = PyPDFLoader(PDF_PATH)
        docs = loader.load()

        logging.info("Splitting text into chunks...")
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = text_splitter.split_documents(docs)

        logging.info("Generating embeddings and storing in FAISS...")
        vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
        vectorstore.save_local(DB_FAISS_PATH)

        logging.info(f"Vectorstore saved at {DB_FAISS_PATH}")

    except Exception as e:
        logging.error(f"Error: {e}")


In [10]:
def load_llm():
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) #gpt-4o-mini
        return llm

In [11]:
from langchain.prompts import ChatPromptTemplate

def load_prompt():
        prompt = """ You need to answer the question in the sentence as same as in the  pdf content. .
        Given below is the context and question of the user.
        context = {context}
        question = {question}
        if the answer is not in the pdf , answer "i donot know what the hell you are asking about"
         """
        prompt = ChatPromptTemplate.from_template(prompt)
        return prompt

In [12]:
#Import Dependencies
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings


def load_knowledgeBasee():
        embeddings=OpenAIEmbeddings()
        DB_FAISS_PATH = 'vectorstore/db_faiss'
        db = FAISS.load_local(DB_FAISS_PATH, embeddings,allow_dangerous_deserialization=True)
        return db

In [13]:
knowledgeBase=load_knowledgeBasee()
llm=load_llm()
prompt=load_prompt()

In [14]:
def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

In [16]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
query = "who is seeta?"
if True:
    similar_embeddings=knowledgeBase.similarity_search(query)
    similar_embeddings=FAISS.from_documents(documents=similar_embeddings, embedding=OpenAIEmbeddings())

    #creating the chain for integrating llm,prompt,stroutputparser
    retriever = similar_embeddings.as_retriever()
    rag_chain = (
                        {"context": retriever | format_docs, "question": RunnablePassthrough()}
                        | prompt
                        | llm
                        | StrOutputParser()
                    )

    response=rag_chain.invoke(query)

In [17]:
response

'Sita is the wife of Rama and the central female character in the Ramayana.'